# Utility of Functions for Building NEU

---

In [ ]:
# Robustness Parameter
robustness_parameter = 5

---

# Loss Function(s)

In [1]:
def above_percentile(x, p): #assuming the input is flattened: (n,)

    samples = Kb.cast(Kb.shape(x)[0], Kb.floatx()) #batch size
    p =  (100. - p)/100.  #100% will return 0 elements, 0% will return all elements

    #selected samples
    values, indices = tf.math.top_k(x, samples)

    return values

def Robust_MSE_ES(p):
    def ES_p_loss(y_true, y_predicted):
        ses = Kb.pow(y_true-y_predicted,2)
        above = above_percentile(Kb.flatten(ses), p)
        return Kb.mean(above)
    return loss

In [3]:
# @tf.function
# def Robust_MSE(robustness_parameter):
#     # Define Entropic Loss
#     def Entropic_loss(y_true, y_pred):
#         # Compute Exponential Utility
#         loss_out = tf.math.abs((y_true - y_pred))
#         loss_out = tf.math.exp(-robustness_parameter*loss_out)
#         loss_out = tf.math.reduce_sum(loss_out)

#         # Return Value
#         return loss_out
#     return Entropic_loss
# # 
# # def Robust_MSE(y_true, y_pred):
# #     # Compute Exponential Utility
# #     loss_out = tf.math.abs((y_true - y_pred))
# #     loss_out = tf.math.exp(robustness_parameter*loss_out)
# #     loss_out = tf.math.reduce_sum(loss_out)

# #     # Return Value
# #     return loss_out

Generalization Loss-Function

In [ ]:
def Robust_MSE(robustness_parameter):
    # Define Entropic Loss
    def Entropic_loss(y_true, y_pred):
        # Initialize Loss
        absolute_errors_eval = tf.math.abs((y_true - y_pred))
        
        # Compute Exponential        
        loss_out_expweights = tf.math.exp(-robustness_parameter*absolute_errors_eval)
        loss_out_expweights_totals = tf.math.reduce_sum(loss_out_expweights)
        loss_out_weighted = loss_out_expweights/tf.math.reduce_sum(loss_out_expweights)
        loss_out_weighted = loss_out_weighted*absolute_errors_eval
        loss_out_weighted = tf.math.reduce_sum(loss_out_weighted)
        
        # Compute Average Loss
        loss_average = tf.math.reduce_mean(absolute_errors_eval)

        # Return Value
        loss_out = loss_out_weighted - loss_average
        
        return loss_out
    return Entropic_loss

# Building NEU
Base Model

In [ ]:
# define and fit the base model
def get_base_model(trainx, trainy, Pre_Epochs_in, depth, height):
    # Define Model
    #----------------#
    # Initialize
    input_layer = tf.keras.Input(shape=[d+D])
    
    # Apply Reconfiguration Unit
    output_layer  = Reconfiguration_unit(height)(input_layer)
    
    if depth > 0:
        output_layer = Shift_Layers(d+D)(output_layer)
        output_layer  = Reconfiguration_unit(height)(output_layer)
        output_layer = Shift_Layers(d+D)(output_layer)
    
    # Output
#     output_layer = projection_layer(output_layer)
    reconfiguration_basic = tf.keras.Model(inputs=[input_layer], outputs=[output_layer])
    
    # Compile Model
    #----------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.Adagrad(learning_rate=10**(-6))
    # Compile
    reconfiguration_basic.compile(loss = 'mae',#Robust_MSE,
                    optimizer = optimizer_on,
                    metrics = ['mse'])
    
    # Fit Model
    #----------------#
    reconfiguration_basic.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)
        
    # Return Output
    return reconfiguration_basic

#### Greedy Initialization of Subsequent Units
Build reconfiguration and pre-train using greedy approach.

In [ ]:
def add_reconfiguration_unit_greedily(model, trainx, trainy, Pre_Epochs_in, depth, height):

    # Dissasemble Network
    layers = [l for l in model.layers]

    # Define new reconfiguration unit to be added
    output_layer_new  = Reconfiguration_unit(d+D)(layers[len(layers)-2].output)

    if depth > 0:
        output_layer_new = Shift_Layers(d+D)(output_layer_new)
        output_layer_new  = Reconfiguration_unit(height)(output_layer_new)
        output_layer_new = Shift_Layers(d+D)(output_layer_new)

    for i in range(len(layers)):
        layers[i].trainable = False


    # build model
    new_model = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()


    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.Adagrad(learning_rate=10**(-6))
    # Compile Model
    new_model.compile(loss = 'mae',#Robust_MSE,
                    optimizer = optimizer_on,
                    metrics = ['mse'])

    # Fit Model
    #----------------#
    new_model.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)

    # Return Output
    return new_model

#### Train and Compile (entire) reconfiguration using greedy-initializations past from previous helper functions.
Train reconfiguration together (initialized by greedy) layer-wise initializations.

In [1]:
def build_reconfiguration(model_greedy_initialized, trainx, trainy, Full_Epochs_in, height):

    # Dissasemble Network
    layers = [l for l in model_greedy_initialized.layers]

    # Define new reconfiguration unit to be added
    output_layer_new  = Shift_Layers(d+D)(layers[len(layers)-2].output)
    output_layer_new  = Reconfiguration_unit(height)(output_layer_new)
    output_layer_new  = Shift_Layers(d+D)(output_layer_new)

    # Output Layer
#     output_layer_new = projection_layer(output_layer_new)

    for i in range(len(layers)):
        layers[i].trainable = True


    # build model
    reconfiguration = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()



    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.Adagrad(learning_rate=10**(-4))

    # Compile Model
    reconfiguration.compile(loss = Robust_MSE(.3),
                    optimizer = optimizer_on,
                    metrics = ['mse','mae'])

    # Fit Model
    #----------------#
    reconfiguration.fit(trainx, trainy, epochs=Full_Epochs_in, verbose=1)

    # Return Output
    return reconfiguration